# 🔬 Phase 1: Ablation Study — Where is Information Lost?

## Goal
Understand exactly WHERE the 37% gap comes from:
- Pair features themselves?
- Random projection?
- Ternary quantization?
- Combination?

## Experiments
| Config | Description | Dims |
|--------|-------------|------|
| A | 1536-float (no HDC) | 1536 |
| B | 4096-float (projection only) | 4096 |
| C | 1536-ternary (quantization only) | 1536 |
| D | 4096-ternary (full HDC) | 4096 |

---

In [ ]:
!pip install -q sentence-transformers datasets
print("✅ Dependencies installed")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import json

from datasets import load_dataset
from sentence_transformers import SentenceTransformer

print(f"PyTorch: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
print(f"\n🔬 Phase 1: Ablation Study")

In [ ]:
# Load encoder
encoder = SentenceTransformer('all-MiniLM-L6-v2')
SEMANTIC_DIM = 384
print(f"✅ Encoder: {SEMANTIC_DIM}d")

In [ ]:
# Load MNLI
print("Loading MNLI...")
dataset = load_dataset("glue", "mnli")

TRAIN_SIZE = 5000
TEST_SIZE = 500

train_data = dataset['train'].shuffle(seed=42).select(range(TRAIN_SIZE))
test_data = dataset['validation_matched'].select(range(TEST_SIZE))

train_labels = np.array(train_data['label'])
test_labels = np.array(test_data['label'])

print(f"✅ Train: {TRAIN_SIZE}, Test: {TEST_SIZE}")

In [ ]:
# Pre-compute embeddings
print("\n🔄 Computing embeddings...")

train_p = encoder.encode(train_data['premise'], show_progress_bar=True)
train_h = encoder.encode(train_data['hypothesis'], show_progress_bar=True)
test_p = encoder.encode(test_data['premise'], show_progress_bar=True)
test_h = encoder.encode(test_data['hypothesis'], show_progress_bar=True)

print(f"✅ Embeddings: {train_p.shape}")

In [ ]:
# Create pair features (best from previous experiments: diff + prod)
def make_pair_features(p, h, method='full'):
    """Create pair features."""
    if method == 'full':
        # [P, H, P-H, P*H] = 1536d
        return np.concatenate([p, h, p - h, p * h], axis=1)
    elif method == 'diff_prod':
        # [P-H, P*H] = 768d (best from encoding strategies)
        return np.concatenate([p - h, p * h], axis=1)

# Use full features for ablation
train_features = make_pair_features(train_p, train_h, 'full')
test_features = make_pair_features(test_p, test_h, 'full')

print(f"✅ Pair features: {train_features.shape}")

In [ ]:
# Transformation functions
def random_projection(features, dim=4096, seed=42):
    """Random projection to higher dimension."""
    np.random.seed(seed)
    proj = np.random.randn(features.shape[1], dim).astype(np.float32)
    proj /= np.linalg.norm(proj, axis=0, keepdims=True)
    return features @ proj

def ternary_quantize(features):
    """Quantize to {-1, 0, +1}."""
    thr = 0.3 * np.std(features, axis=1, keepdims=True)
    return np.where(features > thr, 1,
                    np.where(features < -thr, -1, 0)).astype(np.float32)

print("✅ Transformation functions ready")

In [ ]:
# Prepare all 4 configurations
HDC_DIM = 4096

configs = {
    'A: 1536-float (no HDC)': {
        'train': train_features,
        'test': test_features,
        'dim': 1536
    },
    'B: 4096-float (projection)': {
        'train': random_projection(train_features, HDC_DIM),
        'test': random_projection(test_features, HDC_DIM),
        'dim': HDC_DIM
    },
    'C: 1536-ternary (quantization)': {
        'train': ternary_quantize(train_features),
        'test': ternary_quantize(test_features),
        'dim': 1536
    },
    'D: 4096-ternary (full HDC)': {
        'train': ternary_quantize(random_projection(train_features, HDC_DIM)),
        'test': ternary_quantize(random_projection(test_features, HDC_DIM)),
        'dim': HDC_DIM
    }
}

print("✅ All configurations prepared:")
for name, cfg in configs.items():
    print(f"   {name}: {cfg['train'].shape}")

In [ ]:
# Dataset and Model
class SimpleDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, num_classes=3, dropout=0.3):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )
    
    def forward(self, x):
        return self.classifier(x)

In [ ]:
def train_and_evaluate(train_features, train_labels, test_features, test_labels,
                       input_dim, num_epochs=20, lr=1e-3, verbose=False):
    """Train MLP and return best accuracy."""
    
    train_dataset = SimpleDataset(train_features, train_labels)
    test_dataset = SimpleDataset(test_features, test_labels)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    
    model = MLPClassifier(input_dim=input_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    best_acc = 0
    history = []
    
    for epoch in range(num_epochs):
        # Train
        model.train()
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            loss = criterion(model(features), labels)
            loss.backward()
            optimizer.step()
        scheduler.step()
        
        # Evaluate
        model.eval()
        all_preds = []
        with torch.no_grad():
            for features, _ in test_loader:
                features = features.to(device)
                preds = torch.argmax(model(features), dim=1)
                all_preds.extend(preds.cpu().numpy())
        
        acc = accuracy_score(test_labels, all_preds)
        history.append(acc)
        best_acc = max(best_acc, acc)
        
        if verbose and (epoch + 1) % 5 == 0:
            print(f"  Epoch {epoch+1}: {acc:.1%}")
    
    return best_acc, history

In [ ]:
print("\n" + "="*60)
print("🔬 RUNNING ABLATION STUDY")
print("="*60)

results = {}

for name, cfg in configs.items():
    print(f"\n📊 Testing: {name}")
    
    acc, history = train_and_evaluate(
        cfg['train'], train_labels,
        cfg['test'], test_labels,
        input_dim=cfg['dim'],
        num_epochs=25,
        verbose=True
    )
    
    results[name] = {
        'accuracy': acc,
        'dim': cfg['dim'],
        'history': history
    }
    
    print(f"   ✅ Best: {acc:.1%}")

In [ ]:
print("\n" + "="*60)
print("📊 ABLATION RESULTS")
print("="*60)

# Extract accuracies
acc_A = results['A: 1536-float (no HDC)']['accuracy']
acc_B = results['B: 4096-float (projection)']['accuracy']
acc_C = results['C: 1536-ternary (quantization)']['accuracy']
acc_D = results['D: 4096-ternary (full HDC)']['accuracy']

print(f"\n{'Config':<35} {'Accuracy':<10} {'vs A (baseline)'}")
print("-" * 60)
print(f"{'A: 1536-float (no HDC)':<35} {acc_A:.1%}      baseline")
print(f"{'B: 4096-float (projection)':<35} {acc_B:.1%}      {(acc_B - acc_A)*100:+.1f}%")
print(f"{'C: 1536-ternary (quantization)':<35} {acc_C:.1%}      {(acc_C - acc_A)*100:+.1f}%")
print(f"{'D: 4096-ternary (full HDC)':<35} {acc_D:.1%}      {(acc_D - acc_A)*100:+.1f}%")

print(f"\n📈 LOSS ANALYSIS:")
print(f"   Projection loss (A→B): {(acc_A - acc_B)*100:.1f}%")
print(f"   Quantization loss (A→C): {(acc_A - acc_C)*100:.1f}%")
print(f"   Combined loss (A→D): {(acc_A - acc_D)*100:.1f}%")
print(f"   Interaction (B+C expected vs D actual): {(acc_B + acc_C - acc_A - acc_D)*100:.1f}%")

In [ ]:
# Determine bottleneck
proj_loss = acc_A - acc_B
quant_loss = acc_A - acc_C
total_loss = acc_A - acc_D

print("\n" + "="*60)
print("🎯 BOTTLENECK IDENTIFICATION")
print("="*60)

if acc_A < 0.60:
    print(f"\n⚠️ BASELINE (A) IS LOW: {acc_A:.1%}")
    print(f"   The pair features themselves don't contain enough info!")
    print(f"   → Need better encoder or richer features")
    bottleneck = "encoder/features"
elif quant_loss > proj_loss * 2:
    print(f"\n⚠️ QUANTIZATION is the main bottleneck")
    print(f"   Projection loss: {proj_loss*100:.1f}%")
    print(f"   Quantization loss: {quant_loss*100:.1f}%")
    print(f"   → Try softer quantization or higher dimensions")
    bottleneck = "quantization"
elif proj_loss > quant_loss * 2:
    print(f"\n⚠️ PROJECTION is the main bottleneck")
    print(f"   Projection loss: {proj_loss*100:.1f}%")
    print(f"   Quantization loss: {quant_loss*100:.1f}%")
    print(f"   → Try learned projection or different architecture")
    bottleneck = "projection"
else:
    print(f"\n📊 BOTH contribute roughly equally")
    print(f"   Projection loss: {proj_loss*100:.1f}%")
    print(f"   Quantization loss: {quant_loss*100:.1f}%")
    print(f"   → Need to address both")
    bottleneck = "both"

In [ ]:
# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
ax = axes[0]
names = ['A: Float\n(baseline)', 'B: Float\n+Projection', 'C: Ternary\n(no proj)', 'D: Ternary\n+Projection']
accs = [acc_A, acc_B, acc_C, acc_D]
colors = ['green', 'lightblue', 'orange', 'lightcoral']
bars = ax.bar(names, accs, color=colors, edgecolor='black')
ax.axhline(y=0.33, color='gray', linestyle='--', alpha=0.5, label='Random (33%)')
ax.set_ylabel('Accuracy')
ax.set_title('Ablation Study: Where is Information Lost?')
ax.set_ylim(0.3, max(accs) + 0.1)
for bar, acc in zip(bars, accs):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{acc:.1%}', ha='center', fontweight='bold', fontsize=12)
ax.legend()

# Loss breakdown
ax = axes[1]
losses = ['Projection\n(A→B)', 'Quantization\n(A→C)', 'Combined\n(A→D)']
loss_vals = [proj_loss * 100, quant_loss * 100, total_loss * 100]
colors = ['lightblue', 'orange', 'lightcoral']
bars = ax.bar(losses, loss_vals, color=colors, edgecolor='black')
ax.set_ylabel('Accuracy Loss (%)')
ax.set_title('Information Loss by Component')
ax.axhline(y=0, color='black', linewidth=0.5)
for bar, val in zip(bars, loss_vals):
    y_pos = bar.get_height() + 0.5 if val >= 0 else bar.get_height() - 1.5
    ax.text(bar.get_x() + bar.get_width()/2, y_pos,
            f'{val:.1f}%', ha='center', fontweight='bold', fontsize=12)

plt.tight_layout()
plt.savefig('ablation_study_results.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Learning curves
fig, ax = plt.subplots(figsize=(10, 6))

for name, data in results.items():
    short_name = name.split(':')[0]
    ax.plot(data['history'], label=f"{short_name}: {data['accuracy']:.1%}", linewidth=2)

ax.axhline(y=0.33, color='gray', linestyle='--', alpha=0.5, label='Random')
ax.set_xlabel('Epoch')
ax.set_ylabel('Test Accuracy')
ax.set_title('Learning Curves: Ablation Study')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('ablation_learning_curves.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Save results
output = {
    'experiment': 'Phase 1: Ablation Study',
    'dataset': 'MNLI',
    'train_size': TRAIN_SIZE,
    'test_size': TEST_SIZE,
    'encoder': 'all-MiniLM-L6-v2',
    'hdc_dim': HDC_DIM,
    'results': {
        'A_float_baseline': acc_A,
        'B_float_projected': acc_B,
        'C_ternary_only': acc_C,
        'D_ternary_projected': acc_D
    },
    'loss_analysis': {
        'projection_loss': float(proj_loss),
        'quantization_loss': float(quant_loss),
        'combined_loss': float(total_loss)
    },
    'bottleneck': bottleneck,
    'timestamp': datetime.now().isoformat()
}

with open('ablation_study_results.json', 'w') as f:
    json.dump(output, f, indent=2)

print("\n✅ Results saved!")
print(json.dumps(output, indent=2))

In [ ]:
print("\n" + "="*60)
print("📋 NEXT STEPS BASED ON RESULTS")
print("="*60)

if bottleneck == "encoder/features":
    print("""
🔴 ENCODER/FEATURES is the bottleneck!

Recommended next experiments:
1. Try NLI-tuned encoder (nli-mpnet-base-v2)
2. Try two-vector approach (separate HDC for P and H)
3. Try richer features (add more interactions)
    """)
elif bottleneck == "quantization":
    print("""
🟠 QUANTIZATION is the main bottleneck!

Recommended next experiments:
1. Try 5-level quantization {-2,-1,0,1,2}
2. Try adaptive thresholds
3. Try higher dimensions before quantization
4. Try tanh before quantization
    """)
elif bottleneck == "projection":
    print("""
🔵 PROJECTION is the main bottleneck!

Recommended next experiments:
1. Try learned projection (train on MNLI)
2. Try Project-First-Bind-Later architecture
3. Try circular convolution instead of random projection
    """)
else:
    print("""
🟡 BOTH projection and quantization contribute!

Recommended next experiments:
1. Try Project-First-Bind-Later (addresses both)
2. Try two-vector approach
3. Try NLI encoder + higher dimensions
    """)

In [ ]:
from google.colab import files
files.download('ablation_study_results.json')
files.download('ablation_study_results.png')
files.download('ablation_learning_curves.png')